In [ ]:
!pip install pycuda

In [2]:
import numpy as np
from timeit import default_timer as timer
import pycuda.autoinit
from pycuda.driver import In, Out, Context
from pycuda.compiler import SourceModule

BLOCK_SIZE = 32
MATRIX_SIZES = [128, 256, 512, 1024, 2048]
BLOCK = (BLOCK_SIZE, BLOCK_SIZE, 1)

kernel = SourceModule(
    """
    __global__ void matMul(float* a, float* b, float* c, int* N){
        const int blockSize = %(BLOCK_SIZE)s;
        int n = N[0];
        int bx = blockIdx.x,
            by = blockIdx.y,
            tx = threadIdx.x,
            ty = threadIdx.y;
        int aStart = n * blockSize * by,
            bStart = blockSize * bx;
        int aStep = blockSize,
            bStep = blockSize * n;
        int blockCount = gridDim.x;
        float sum = 0.0f;
        int ia = aStart,
            ib = bStart;

        __shared__ float as[blockSize][blockSize];
        __shared__ float bs[blockSize][blockSize];
        for (int i = 0; i < blockCount; i++){
          as[ty][tx] = a[ia + n * ty + tx];
          bs[ty][tx] = b[ib + n * ty + tx];
          __syncthreads ();
          for ( int k = 0; k < blockSize; k++){
            sum += as[ty][k] * bs[k][tx];
          }
          __syncthreads ();
          ia += aStep;
          ib += bStep;
        }
        c[n * blockSize * by + blockSize * bx + n * ty + tx] = sum;
    }
    """ % {
        'BLOCK_SIZE' : BLOCK_SIZE
    }
)

matMul = kernel.get_function("matMul")


def gpu(a, b, n):
    N = np.array([n])
    c  = np.zeros_like(a, dtype=np.float32)
    grid_dim = (n // BLOCK_SIZE, n // BLOCK_SIZE)
    matMul(In(a), In(b), Out(c), In(N), block=BLOCK, grid=grid_dim)
    Context.synchronize()
    return c


def cpu(a, b):
    return a.dot(b)


def test(a, b, n):
    start = timer()
    cpu_res = cpu(a, b)
    cpu_multiply_time = timer() - start

    start = timer()
    gpu_res = gpu(a, b, n)
    gpu_multiply_time = timer() - start

    return cpu_multiply_time * 1000, gpu_multiply_time * 1000, np.allclose(cpu_res, gpu_res)

In [3]:
TEST_ROUND = 20

print("Size | CPU time, ms | GPU time, ms | Speedup")

for size in MATRIX_SIZES:
    cpu_time = 0
    gpu_time = 0

    for i in range(TEST_ROUND):
        A = np.array(np.random.rand(size, size), dtype=np.float32)
        B = np.array(np.random.rand(size, size), dtype=np.float32)
        c_time, g_time, e = test(A, B, size)
        cpu_time += c_time
        gpu_time += g_time

        if not e:
            print("Size = {:d}, round = {:d}: results not equals".format(size, i))

    print("{:4d} | {:12.3f} | {:12.3f} | {:7.2f}".format(size, cpu_time / TEST_ROUND, gpu_time / TEST_ROUND, cpu_time / gpu_time))

Size | CPU time, ms | GPU time, ms | Speedup
 128 |        0.293 |        0.661 |    0.44
 256 |        0.638 |        0.899 |    0.71
 512 |        4.608 |        2.926 |    1.57
1024 |       35.526 |       11.086 |    3.20
2048 |      266.146 |       59.547 |    4.47
